In [22]:
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = Ollama(model="openchat:latest")


In [16]:
from langchain_core.prompts import ChatPromptTemplate

translation_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI translator bot. When a user ask to translate to english something, you just answer with the translation, with no further details, also do not add spaces or other symbols out of the translation"),
        ("human", "Translate 'Ciao, come stai?' to English"),
        ("ai", "Hello, how are you?"),
        ("human", "Translate '{user_input}' to English"),
    ]
)

translation_chain = LLMChain(prompt=translation_template, llm=llm)

In [17]:
translation_chain.run(user_input="Dammi la ricetta per fare i pankakes")

' Give me the recipe for making pancakes'

In [20]:
question_answer_template="""
<<SYS>>You are a helpful assistant, please answer this question being respectful. Provide always concise responses<</SYS>>

[INST]User: {input}
AI: 
[/INST]
"""
question_answer_prompt = PromptTemplate.from_template(question_answer_template)

question_answer_chain = LLMChain(prompt=question_answer_prompt, llm=llm)

In [21]:
from langchain.chains import SimpleSequentialChain, SequentialChain
full_chain = SimpleSequentialChain(chains=[translation_chain, question_answer_chain], verbose=True)

In [22]:
full_chain.run("Dammi la ricetta dei pankakes")



> Entering new SimpleSequentialChain chain...
 Give me the pancake recipe
 Certainly! Here is a simple and delicious pancake recipe for you:

Ingredients:
- 1 cup all-purpose flour
- 2 tablespoons sugar
- 1 teaspoon baking powder
- 1/2 teaspoon salt
- 1 cup milk
- 1 egg
- 2 tablespoons melted butter
- Optional: Add-ins like blueberries, chocolate chips, or nuts

Instructions:
1. In a large bowl, whisk together the flour, sugar, baking powder, and salt.
2. In a separate bowl, beat the egg and then mix in the milk and melted butter.
3. Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix; some lumps are okay.
4. If using add-ins like blueberries or chocolate chips, gently fold them into the batter.
5. Heat a non-stick skillet or griddle over medium heat. Add a small amount of butter or cooking spray to prevent sticking.
6. Pour 1/4 cup portions of the batter onto the heated surface and cook until bubbles form on the surface and the e

' Certainly! Here is a simple and delicious pancake recipe for you:\n\nIngredients:\n- 1 cup all-purpose flour\n- 2 tablespoons sugar\n- 1 teaspoon baking powder\n- 1/2 teaspoon salt\n- 1 cup milk\n- 1 egg\n- 2 tablespoons melted butter\n- Optional: Add-ins like blueberries, chocolate chips, or nuts\n\nInstructions:\n1. In a large bowl, whisk together the flour, sugar, baking powder, and salt.\n2. In a separate bowl, beat the egg and then mix in the milk and melted butter.\n3. Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix; some lumps are okay.\n4. If using add-ins like blueberries or chocolate chips, gently fold them into the batter.\n5. Heat a non-stick skillet or griddle over medium heat. Add a small amount of butter or cooking spray to prevent sticking.\n6. Pour 1/4 cup portions of the batter onto the heated surface and cook until bubbles form on the surface and the edges start to look set (about 2 minutes).\n7. Flip the pa

In [23]:
recognizing_language_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI translator bot. When a user ask to recognize the language of a message you just answer with the language name as output, with no further details, also do not add spaces or other symbols out of the translation"),
        ("human", "Recognize the language of 'Ciao, come stai?'"),
        ("ai", "Italian"),
        ("human", "Recognize the language of '{user_input}'"),
        ("ai", "{language}"),
    ], 

)
recognizing_language_chain = LLMChain(prompt=translation_template, llm=llm)

In [ ]:
full_chain = SimpleSequentialChain(chains=[translation_chain, question_answer_chain], verbose=True)

# aPI CHAIN

In [23]:
from langchain.chains import APIChain
from langchain.chains.api import open_meteo_docs
print(open_meteo_docs.OPEN_METEO_DOCS)
chain_new = APIChain.from_llm_and_api_docs(llm, open_meteo_docs.OPEN_METEO_DOCS, verbose=True, limit_to_domains=None)

BASE URL: https://api.open-meteo.com/

API Documentation
The API endpoint /v1/forecast accepts a geographical coordinate, a list of weather variables and responds with a JSON hourly weather forecast for 7 days. Time always starts at 0:00 today and contains 168 hours. All URL parameters are listed below:

Parameter	Format	Required	Default	Description
latitude, longitude	Floating point	Yes		Geographical WGS84 coordinate of the location
hourly	String array	No		A list of weather variables which should be returned. Values can be comma separated, or multiple &hourly= parameter in the URL can be used.
daily	String array	No		A list of daily weather variable aggregations which should be returned. Values can be comma separated, or multiple &daily= parameter in the URL can be used. If daily weather variables are specified, parameter timezone is required.
current_weather	Bool	No	false	Include current weather conditions in the JSON output.
temperature_unit	String	No	celsius	If fahrenheit is set, al

In [25]:
chain_new.run("What is the temperature like right now in Bedok, Singapore in degrees celsius?")



> Entering new APIChain chain...
 https://api.open-meteo.com/v1/forecast?latitude=1.3521&longitude=103.9194&current_weather=true&temperature_unit=celsius
{"latitude":1.375,"longitude":103.875,"generationtime_ms":0.04291534423828125,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":29.0,"current_weather_units":{"time":"iso8601","interval":"seconds","temperature":"°C","windspeed":"km/h","winddirection":"°","is_day":"","weathercode":"wmo code"},"current_weather":{"time":"2024-03-02T18:30","interval":900,"temperature":26.0,"windspeed":8.7,"winddirection":24,"is_day":0,"weathercode":3}}

> Finished chain.


' The current temperature in Bedok, Singapore is 26 degrees Celsius.'